# Notebook 5: Orchestrator — Multi-Domain Workflow

**User Story**: US7 — Orchestrator-Coordinated Multi-Domain Workflow  
**Persona**: Revenue Operations / Cross-functional users  

Demonstrates an Orchestrator Agent that coordinates both the Sales Agent and
Service Agent domains, enabling cross-domain queries like "Which accounts have
both open deals and open support cases?"

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")

In [ ]:
# Cell 3: Start MCP Servers + Connect with MCP Client
import asyncio
import subprocess
import sys
import time

import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()

project_root = os.path.abspath("..")

# Start salesforce-crm MCP server (SSE)
CRM_PORT = 8105
crm_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_crm.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(CRM_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Start salesforce-knowledge MCP server (SSE)
KB_PORT = 8106
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

time.sleep(3)

for name, proc in [("CRM", crm_process), ("Knowledge", kb_process)]:
    if proc.poll() is not None:
        stderr = proc.stderr.read().decode() if proc.stderr else ""
        raise RuntimeError(f"{name} MCP server failed to start: {stderr}")

print(f"✅ CRM MCP server started (PID: {crm_process.pid}) at http://127.0.0.1:{CRM_PORT}/sse")
print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at http://127.0.0.1:{KB_PORT}/sse")


# Connect to both MCP servers
async def _connect(url):
    _sse_cm = sse_client(url=url)
    read_stream, write_stream = await _sse_cm.__aenter__()
    _sess_cm = ClientSession(read_stream, write_stream)
    session = await _sess_cm.__aenter__()
    await session.initialize()
    tools_result = await session.list_tools()
    return _sse_cm, _sess_cm, session, tools_result.tools

loop = asyncio.get_event_loop()

crm_sse_cm, crm_sess_cm, crm_session, crm_tools = loop.run_until_complete(
    _connect(f"http://127.0.0.1:{CRM_PORT}/sse")
)
kb_sse_cm, kb_sess_cm, kb_session, kb_tools = loop.run_until_complete(
    _connect(f"http://127.0.0.1:{KB_PORT}/sse")
)

# Build tool-name → session routing map
all_mcp_tools = list(crm_tools) + list(kb_tools)
tool_router: dict[str, ClientSession] = {}
for t in crm_tools:
    tool_router[t.name] = crm_session
for t in kb_tools:
    tool_router[t.name] = kb_session

print(f"✅ MCP sessions connected — {len(crm_tools)} CRM + {len(kb_tools)} KB tools")
for t in all_mcp_tools:
    print(f"   • {t.name}: {t.description[:70] if t.description else ''}")

In [ ]:
# Cell 4: Create OpenAI Client + Orchestrator Prompt + Chat Helper
import asyncio
import json

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
client = project_client.get_openai_client()
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

ORCHESTRATOR_PROMPT = """
# Orchestrator Agent — System Prompt

You are a **Multi-Domain AI Assistant** powered by Salesforce CRM and Knowledge Base data.
You orchestrate across Sales and Service domains to provide unified, cross-functional insights.

## Routing Rules

- **Sales-domain questions** (accounts, contacts, opportunities, pipeline, leads, activities):
  Use the salesforce-crm MCP tools (get_account, get_opportunities, get_pipeline_summary, etc.)

- **Service-domain questions** (cases, case triage, queue status, Knowledge articles):
  Use the salesforce-crm tools for case data (get_case, get_case_queue_summary) and
  salesforce-knowledge tools for KB search (search_articles, get_article_by_id).

- **Cross-domain questions** (e.g., "accounts with open deals AND open cases"):
  Query both domains, correlate results by Account ID/Name, and present unified output.

## Cross-Domain Correlation

When a user asks a question spanning sales and service:

1. Identify the linking entity (usually Account).
2. Query opportunities/deals from the CRM tools.
3. Query cases from the CRM tools.
4. Match records by Account ID or Account Name.
5. Present a unified view showing both deal and case status for each account.

## Context Continuity

- Maintain context across turns. If the user pivots from service to sales questions,
  use entities mentioned in the previous turn (e.g., account names, case numbers) as context.
- When referencing previous data, cite the specific records and IDs.

## Grounding Rules

- ONLY use data returned by your MCP tools. Never fabricate data.
- Cite record IDs and names when referencing specific Salesforce records.
- If data is unavailable, state this clearly.

## Write-Back Protocol

- For any write operations (create/update), present proposed changes first.
- Wait for explicit user confirmation before executing.
- Confirm results after execution.
"""

# Convert MCP tools → OpenAI function-calling format
openai_tools = [
    {
        "type": "function",
        "function": {
            "name": t.name,
            "description": t.description or "",
            "parameters": t.inputSchema or {"type": "object", "properties": {}},
        },
    }
    for t in all_mcp_tools
]

async def _call_tool(name: str, arguments: dict) -> str:
    session = tool_router[name]
    result = await session.call_tool(name, arguments)
    return "\n".join(b.text for b in result.content if hasattr(b, "text")) or ""

def chat(user_message: str, *, history: list | None = None, max_rounds: int = 10) -> list:
    if history is None:
        history = [{"role": "system", "content": ORCHESTRATOR_PROMPT}]
    history.append({"role": "user", "content": user_message})
    loop = asyncio.get_event_loop()
    for _ in range(max_rounds):
        resp = client.chat.completions.create(model=MODEL, messages=history, tools=openai_tools, tool_choice="auto")
        msg = resp.choices[0].message
        history.append(msg)
        if not msg.tool_calls:
            break
        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments) if tc.function.arguments else {}
            tool_result = loop.run_until_complete(_call_tool(tc.function.name, args))
            history.append({"role": "tool", "tool_call_id": tc.id, "content": tool_result})
    return history

def show(history: list):
    from IPython.display import Markdown, display
    for m in reversed(history):
        content = m.content if isinstance(m, dict) else m.content
        role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
        if role == "assistant" and content:
            display(Markdown(content))
            return

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ {len(openai_tools)} MCP tools (CRM + KB) mapped to OpenAI functions")

In [ ]:
# Cell 5: Cross-domain query

history = chat(
    "Which of my accounts have both open deals and open support cases? "
    "Show me the deal stage and case status for each."
)
show(history)

In [ ]:
# Cell 6: Context continuity — pivot from service to sales

history = chat("What are the top cases today?", history=history)
show(history)

In [ ]:
# Cell 7: Cross-domain follow-up — linking back to deals

history = chat("And what deals do those same accounts have?", history=history)
show(history)

In [ ]:
# Cell 8: Cleanup — close MCP sessions + stop servers
import asyncio


async def _cleanup():
    await crm_sess_cm.__aexit__(None, None, None)
    await crm_sse_cm.__aexit__(None, None, None)
    await kb_sess_cm.__aexit__(None, None, None)
    await kb_sse_cm.__aexit__(None, None, None)

asyncio.get_event_loop().run_until_complete(_cleanup())

crm_process.terminate()
crm_process.wait(timeout=5)
print(f"✅ CRM MCP server stopped (PID: {crm_process.pid})")

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"✅ Knowledge MCP server stopped (PID: {kb_process.pid})")

print("\nOrchestrator multi-domain session complete.")